In [7]:
import pandas as pd
import spacy 
nlp = spacy.load("de_core_news_sm")
import matplotlib.pyplot as plt
import gzip,os,glob
import numpy as np
import pickle
import librosa
from sklearn import *
from math import floor

In [8]:
def generate_random_vector(dimensions,seed):
    arr = np.array([0] * (dimensions-seed) + [1] * int(seed/2) + [-1] * int(seed/2) )
    np.random.shuffle(arr)
    return(arr)

In [9]:
import csv
import re
textz=[]
word_vec=dict()
word_docz=dict()
dimensions=500
seed=8

with open('../data/after_nltk/df_content_after_nltk+titles.csv', newline='', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile) 
    for row in reader:
        textz.append(row['content'].lower())
        #if int(row['id'])>5000:
        #    break
        if int(row['id'])%1000==0:
            print(row['id'])
        current_doc_wordz=dict()
        content=row['content'].lower()
        for word in re.split('[\W]', content):
            if word not in current_doc_wordz:
                current_doc_wordz[word]=1
                try:
                    word_docz[word]=word_docz[word]+1
                except:
                    word_docz[word]=1
                    word_vec[word]=generate_random_vector(dimensions,seed)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [10]:
#we focus on words which occur in more than HAPAX_THRESHOLD and less than STOPWORD_THRESHOLD documents because they just bring noise in our vector space

HAPAX_THRESHOLD=4
STOPWORD_THRESHOLD=1000
nonsingular_wordz = dict((k, v) for k, v in word_docz.items() if v > HAPAX_THRESHOLD and v<STOPWORD_THRESHOLD)


#we create IDF (inverse-document frequency) weights
normalize_factor=1 / sum(nonsingular_wordz.values())
for k in nonsingular_wordz:
    nonsingular_wordz[k]*=normalize_factor
    #print(nonsingular_wordz[k],k)
    
len(nonsingular_wordz)

24213

In [11]:
#0th pass - from random word vectors to meaningful doc vectors
print("rand word 2 doc\n")
doc_vec=np.zeros((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split(' ',text):
        if word in nonsingular_wordz:
            #document vector is a linear combination of IDF-weighted word vectors which document contains
            doc_vec[line_id] = ( word_vec[word] / (nonsingular_wordz[word]) ) + doc_vec[line_id]
            #simplified version (withoug IDF-weighting)
            #doc_vec[line_id] = word_vec[word]   + doc_vec[line_id]
    line_id=line_id+1
 

for i in range(len(doc_vec)):
    doc_vec[i]=preprocessing.normalize(doc_vec[i].reshape(1,-1))

    
#from document vectors to word vectors
print("doc 2 word")
word_vec2=dict()
line_id=0
for text in textz:
    for word in re.split(' ',text):
        if word in nonsingular_wordz:
            #print(word)
            if word in word_vec2:
                word_vec2[word] +=  doc_vec[line_id] 
            else:
                word_vec2[word]=np.array([0.0]*dimensions)
                #print(str(word_vec2[word]))

                #doc_vec[line_id][0:18149]
    line_id=line_id+1

    
#normalize all word vectors
for k in word_vec2.keys():
    word_vec2[k]=preprocessing.normalize(word_vec2[k].reshape(1,-1))

print("word 2 doc")  
#and to doc vectors again
doc_vec2=np.zeros((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split('[\W]',text):
        if word in nonsingular_wordz:
            #document vector is a linear combination of IDF-weighted word vectors which document contains
            try:
                doc_vec2[line_id] = ( word_vec2[word] / (nonsingular_wordz[word]) ) + doc_vec2[line_id]
            except:
                1
    line_id=line_id+1
    

rand word 2 doc

doc 2 word
word 2 doc


In [12]:
for i in range(len(doc_vec)):
    doc_vec2[i]=preprocessing.normalize(doc_vec2[i].reshape(1,-1))

    
#from document vectors to word vectors
print("doc2word 2")
word_vec2=dict()
line_id=0
for text in textz:
    for word in re.split(' ',text):
        if word in nonsingular_wordz:
            #print(word)
            if word in word_vec2:
                word_vec2[word] +=  doc_vec2[line_id] 
            else:
                word_vec2[word]=np.array([0.0]*dimensions)
                #print(str(word_vec2[word]))

                #doc_vec[line_id][0:18149]
    line_id=line_id+1

    
#normalize all word vectors
for k in word_vec2.keys():
    word_vec2[k]=preprocessing.normalize(word_vec2[k].reshape(1,-1))

print("word 2 doc 2")  
#and to doc vectors again
doc_vec2=np.zeros((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split('[\W]',text):
        if word in nonsingular_wordz:
            #document vector is a linear combination of IDF-weighted word vectors which document contains
            try:
                doc_vec2[line_id] = ( word_vec2[word] / (nonsingular_wordz[word]) ) + doc_vec2[line_id]
            except:
                1
    line_id=line_id+1
    

doc2word 2
word 2 doc 2


In [13]:
from scipy.spatial import distance

def nearest_neighbor(query_id,database):
    query=database[query_id]
    closest = distance.cdist([query], database, 'cosine')
    return closest

In [19]:
import numpy.ma as ma

query_id=5
nns=nearest_neighbor(query_id,doc_vec)
nn_id=np.argmin(ma.masked_where(nns<0.0000000001, nns))
print(query_id,textz[query_id])
print(nn_id,nns[0][nn_id],textz[nn_id])

5 geotechnik bodenmechanik buch vermitteln wichtig aspekte ueber aufbau eigenschaften bodens planung berechnung sowie begutachtung schaeden systems bauwerkbaugrund beruecksichtigen schwerpunkte baugrunderkundung ermittlung bodenkennwerten labor sowie behandlung setzungs tragfaehigkeitsnachweisen einschliesslich erddrucks unterstuetzung verstaendnisses dienen zahlreiche beispiele nachvollziehbar erlaeutert darstellungen basieren aktuell technisch regelwerk buch unverzichtbar orientierungshilfe taeglichen planungs gutachterpraxis
2607 0.5344274399162543 äcker wirte gaben ländlich bodenmarkt liberale eigentumsordnung westfalen 19 jahrhunderts jahrhundert werden preussen baeuerliche eigentumsrechte schaffen entgegen annahmen reformer fuehrte marktgesteuerten bewegung faktors boden derjenige betrieben effizient nutzen acker wandern gut wirt vielmehr bleiben mobilitaet bodens einbetten system erbschaft familiaerer bergabe georg fertigen verfolgen buch bewegung parzellen hoefen landwirten dre

<h1>Compute Cosine Similarity through dot product</h1>

In [15]:
from sklearn.preprocessing import Normalizer
nrm = Normalizer('l2')

doc_vec_normed = nrm.fit_transform(doc_vec)

In [16]:
df = pd.read_pickle('../data/after_nltk/data_after_nltk.pkl')

In [17]:
# choose example 
example = query_id
title_1 = df['title'].iloc[example]
abstract_1 = textz[example]
# get example vector
vec_1 = doc_vec_normed[example,:]

# compute similarity
sim = np.dot(doc_vec_normed[example+1:,:],vec_1)

# find most similar document, see paper
title_2 = df['title'].iloc[np.argmax(sim)+example+1]
abstract_2 = textz[np.argmax(sim)+example+1]

print('\nSimilarity:')
print(np.max(sim))
print('\nTitle 1:')
print(title_1)
print('\nAbstract 1:')
print(abstract_1)
print('\nTitle 2:')
print(title_2)
print('\nAbstract 2:')
print(abstract_2)


Similarity:
0.6897351250877295

Title 1:
 schulbucharbeit das geschichtslehrbuch in der unterrichtspraxis

Abstract 1:
schulbucharbeit geschichtslehrbuch unterrichtspraxis trotzen zunehmend konkurrenz gelten schulbuch leitmedium geschichtsunterrichts manchmal behaupten modern schulbuecher selbstlaeufer verfuegen ueber zahlreiche methodenseiten bieten tipps unterrichtsgestaltung gut geschichtsunterricht moeglich lehrerinnen lehrer intensiv medial eigenlogik schulbuchs auseinandersetzen entstehen schulbuecher aufbauen zeichnen gutes schulbuch methoden schulbucharbeit eignen fuer reflektieren lehrlernprozesse vorliegend band geben zentrale fragen antwort erste umfassend monografie thema

Title 2:
 guter geschichtsunterricht grundlagen erkenntnisse hinweise

Abstract 2:
gut geschichtsunterricht grundlagen erkenntnisse hinweise studie gut geschichtsunterricht grundlagen erkenntnisse hinweise liefern beitrag nutzerorientierten geschichtsdidaktischen unterrichtsforschung zentrum arbeit stehe

<h1>K nearest Neighbors</h1>

In [20]:
# compares similarity of two abstracts through dot product

# validation_vector1 
validation_vector1 = query_id
val_abstract_1 = textz[validation_vector1]
# get validation_vector1
val_vec_1 = doc_vec_normed[validation_vector1,:]

# compute similarity of all entities
vec_temp = []
for vec in doc_vec_normed:
    i = np.dot(val_vec_1,vec)
    vec_temp.append(i)

# turn list into DataFrame
df_vec_temp = pd.DataFrame(vec_temp)

# Create new DataFrame for sorted sim-values
df_vec_temp_sorted = df_vec_temp.copy()
df_vec_temp_sorted.sort_values(by=[0],inplace = True, ascending=False)

# Get list of indices
lst_vec_temp_sorted = df_vec_temp_sorted.iloc[1:11,:].index.values.tolist()
lst_vec_temp_sorted

print(df['title'].iloc[validation_vector1])
print('\n')

for i in lst_vec_temp_sorted:
#    print('\n')
    print(df_vec_temp.iloc[i].values)
    print(df['title'].iloc[i])
#     print(textz[i])
#     print('\n')

 geotechnik bodenmechanik


[0.46557256]
 äcker wirte gaben ländlicher bodenmarkt und liberale eigentumsordnung im westfalen des 19 jahrhunderts
[0.41464291]
 verkehrsmedizin fahreignung fahrsicherheit unfallrekonstruktion madea verkehrsmedizin 2a 
[0.32349903]
 bodenkundliches praktikum eine einführung in pedologisches arbeiten für ökologen land und forstwirte geo und umweltwissenschaftler
[0.29180923]
 umgebungseinflüsse auf antennen aufbauhöhe bodenparameter topographie und vegetation
[0.28118623]
 diagnostik schulischer lern und leistungsschwierigkeiten ein leitfaden
[0.27117204]
 untersuchungen zur entwicklung eines ertragsmesssystems für mähwerke auf basis des leistungsbedarfs eines schwadversetzers
[0.23591744]
 bewertung von boden und fels auf verklebungen und feinkornfreisetzung beim maschinellen tunnelvortrieb
[0.23173468]
 wärmepumpen fehler vermeiden bei planung installation und betrieb
[0.20348628]
 catia v5 einstieg und effektives arbeiten
[0.19869564]
 byzanz historischk

In [21]:
np.save('../data/after_nltk/doc_vec-200.npy', doc_vec)